## RECURRENT NEURAL NETWORKS USING LSTMS

In [98]:
from google.colab import drive
drive.mount('/content/drive')

Importing The Required Libraries

In [99]:
import pandas as pd;
import numpy as np;
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

Loading The Preprocessed Dataset

In [100]:
dataset = pd.read_csv("/content/drive/MyDrive/RNN_Preprocessed_Dataset.csv")
dataset.head()
dataset.drop(columns=["Unnamed: 0","Date"],inplace=True)

Split Into Train & Test

In [101]:
X_train = []
Y_train = []
for i in range(40,len(dataset)):
    X_train.append(dataset.iloc[i-40:i])
    Y_train.append(dataset.iloc[i])
X_train,Y_train = np.array(X_train), np.array(Y_train)

3432 3392 3392


Building The Neural Network

In [93]:
rnn = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],X_train.shape[2])),
    tf.keras.layers.LSTM(units=64,return_sequences=True,dropout=0.1,recurrent_dropout=0.1),
    tf.keras.layers.LSTM(units=64,return_sequences=True),
    tf.keras.layers.LSTM(units=32,return_sequences=True),
    # tf.keras.layers.Dropout(0.2),
    # tf.keras.layers.LSTM(units=50),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1,activation="relu"),
])

In [94]:
rnn.compile(optimizer="adam",loss="mean_squared_error")

Callback Function

In [95]:
callback = tf.keras.callbacks.EarlyStopping(patience=3,monitor="loss")

In [96]:
rnn.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_28 (LSTM)                       │ (None, 40, 50)              │          15,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_28 (Dropout)                 │ (None, 40, 50)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_29 (LSTM)                       │ (None, 40, 50)              │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_29 (Dropout)                 │ (None, 40, 50)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_30 (LSTM)                       │ (None, 40, 50)              │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_30 (Dropout)                 │ (None, 40, 50)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_31 (LSTM)                       │ (None, 50)                  │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_31 (Dropout)                 │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 76,251 (297.86 KB)

 Trainable params: 76,251 (297.86 KB)

 Non-trainable params: 0 (0.00 B)

In [102]:
rnn.fit(X_train,Y_train,epochs=100,batch_size=32,callbacks=[callback])

Epoch 1/100
106/106 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - loss: 15446.4170
Epoch 2/100
106/106 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 10792.8262
Epoch 3/100
106/106 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - loss: 14604.8926
Epoch 4/100
106/106 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - loss: 11178.9814
Epoch 5/100
106/106 ━━━━━━━━━━━━━━━━━━━━ 10s 44ms/step - loss: 13094.2100
Epoch 6/100
106/106 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - loss: 10979.8809
Epoch 7/100
106/106 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - loss: 11026.5908
Epoch 8/100
106/106 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - loss: 16776.7168
Epoch 9/100
106/106 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - loss: 15219.8066
Epoch 10/100
106/106 ━━━━━━━━━━━━━━━━━━━━ 6s 43ms/step - loss: 9629.9658
Epoch 11/100
106/106 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - loss: 16839.0625
Epoch 12/100
106/106 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - loss: 10252.6904
Epoch 13/100
106/106 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - loss: 11211.0928
Epoch 14/100
106/106 ━━━━━━━━━━━━━━━━━━━━ 4s 36

In [ ]:
Y_train = Y_train.astype(np.float32)